In [15]:
#imports 

import gymnasium as gym  # Updated import to use gymnasium instead of gym
from gymnasium import spaces  # Import spaces from gymnasium
import random  # To randomly place the player, win, and lose positions
import numpy as np  # For numerical operations and array handling
from IPython.display import clear_output  # To clear the output display
import os  # For operating system-related functions

# Import the PPO algorithm from stable_baselines3
from stable_baselines3 import PPO
# For checking the custom environment
from stable_baselines3.common.env_checker import check_env


In [16]:
# Global constants

#game board values
NOTHING = 0
PLAYER = 1
WIN = 2
LOSE = 3

#action values
UP = 0
DOWN = 1
LEFT = 2
RIGHT = 3

In [17]:
# helper function
def clear_screen():
    clear_output()
    os.system("cls")

# prints out the environment state in a visually appealing way

def pretty_print(state_array, cumulative_reward):
    clear_screen()
    print(f'Cumulative Reward: {cumulative_reward}')
    print()
    for i in range(6):
        for j in range(6):
            print('{:4}'.format(state_array[i*6 + j]), end="")
        print()


In [18]:
# Define the custom environment class inheriting from gymnasium.Env
class BasicEnv(gym.Env):
    def __init__(self):
        super(BasicEnv, self).__init__()

        # Class variable for cumulative reward
        self.cumulative_reward = 0

        # Set initial state for a flattened 6x6 grid
        self.state = [NOTHING] * 36

        # Randomly place the player, win, and lose positions
        self.player_position = random.randrange(0, 36)
        self.win_position = random.randrange(0, 36)
        self.lose_position = random.randrange(0, 36)

        # Ensure win and lose positions do not overlap with each other or with the player position
        while self.win_position == self.player_position:
            self.win_position = random.randrange(0, 36)
        while self.lose_position == self.win_position or self.lose_position == self.player_position:
            self.lose_position = random.randrange(0, 36)

        # Update the state array with the player, win, and lose positions
        self.state[self.player_position] = PLAYER
        self.state[self.win_position] = WIN
        self.state[self.lose_position] = LOSE

        # Convert the state list to a numpy array
        self.state = np.array(self.state, dtype=np.int16)

        # Define the observation space (valid range for observation in the state)
        self.observation_space = spaces.Box(0, 3, [36,], dtype=np.int16)

        # Define the action space (valid actions: UP, DOWN, LEFT, RIGHT)
        self.action_space = spaces.Discrete(4)

    def step(self, action):
        # Placeholder for debugging information
        info = {}
        truncated = False

        # Set default values for done, reward, and the previous player position
        done = False
        reward = -0.01
        previous_position = self.player_position

        # Debug print for action
        print(f"Action taken: {action}")

        # Check for invalid actions
        if action == UP and (self.player_position - 6) < 0:
            action = -1  # Invalid action
        elif action == DOWN and (self.player_position + 6) >= 36:
            action = -1  # Invalid action
        elif action == LEFT and (self.player_position % 6) == 0:
            action = -1  # Invalid action
        elif action == RIGHT and (self.player_position % 6) == 5:
            action = -1  # Invalid action

        # If the action is valid, move the player
        if action == UP:
            self.player_position -= 6
        elif action == DOWN:
            self.player_position += 6
        elif action == LEFT:
            self.player_position -= 1
        elif action == RIGHT:
            self.player_position += 1
        else:
            print("Invalid action taken, skipping this step")
            return self.state, reward, done, truncated, info

        # Check win or lose condition and set reward
        if self.state[self.player_position] == WIN:
            reward = 1.0
            self.cumulative_reward += reward
            done = True
            clear_screen()
            print(f'Cumulative Reward: {self.cumulative_reward}')
            print('WIN !!!')
        elif self.state[self.player_position] == LOSE:
            reward = -1.0
            self.cumulative_reward += reward
            done = True
            clear_screen()
            print(f'Cumulative Reward: {self.cumulative_reward}')
            print("Lose :B")

        # Update the environment state
        if not done:
            self.state[previous_position] = NOTHING
            self.state[self.player_position] = PLAYER

        self.cumulative_reward += reward

        return self.state, reward, done, truncated, info

    def reset(self, seed=None, options=None):
        self.cumulative_reward = 0

        # Set the initial state to a flattened 6x6 grid
        self.state = [NOTHING] * 36

        # Randomly place the player, win, and lose positions
        self.player_position = random.randrange(0, 36)
        self.win_position = random.randrange(0, 36)
        self.lose_position = random.randrange(0, 36)

        # Ensure win and lose positions do not overlap with each other or with the player position
        while self.win_position == self.player_position:
            self.win_position = random.randrange(0, 36)
        while self.lose_position == self.win_position or self.lose_position == self.player_position:
            self.lose_position = random.randrange(0, 36)

        # Update the state array with the player, win, and lose positions
        self.state[self.player_position] = PLAYER
        self.state[self.win_position] = WIN
        self.state[self.lose_position] = LOSE

        # Convert the state list to a numpy array
        self.state = np.array(self.state, dtype=np.int16)

        return self.state, info

    def render(self):
        pretty_print(self.state, self.cumulative_reward)


In [19]:
# Instantiate the environment
env = BasicEnv()

# Check if the environment follows the gymnasium API
check_env(env)

# Create the PPO agent
model = PPO("MlpPolicy", env, verbose=1)

# Train the agent
model.learn(total_timesteps=10000)

# Save the trained agent
model.save("ppo_basic_env")

# Load the trained agent
model = PPO.load("ppo_basic_env")

# Visualize the current state of the environment
env.render()

# Test the agent
obs, info = env.reset()
for i in range(50):
    # Predict the action to take based on the current observation
    action, _states = model.predict(obs)
    obs, rewards, dones, truncate, info = env.step(
        action)  # Take the action in the environment
    env.render()  # Render the environment to visualize the current state
    if dones:  # If the episode is done, break the loop
        break


NameError: name 'info' is not defined

In [20]:
# Test the agent
def play_game(model, env, episodes=1, steps=50):
    for episode in range(episodes):
        obs, info = env.reset()
        env.render()
        done = False
        step = 0
        while not done and step < steps:
            # Predict the action to take based on the current observation
            action, _states = model.predict(obs)
            obs, rewards, done, truncate, info = env.step(
                action)  # Take the action in the environment
            env.render()  # Render the environment to visualize the current state
            step += 1
            if done:
                print(f"Episode {episode + 1} finished after {step} steps")
                break


# Play the game
play_game(model, env)


NameError: name 'model' is not defined